<center><h1>Testing Notebook</h1></center>


## Loading Entities:

 - Power System
 - Ant Colony

In [ ]:
from acopoweropt import system, colony

PSystem = system.PowerSystem(name='s15')
Colony = colony.Colony(n_ants=5,phr_evp_rate=0.25)

## Testing Power System

In [ ]:
# Randomly select an operation
operation = PSystem.sample_operation()

# Solve system
solution = PSystem.solve(operation=operation)

In [ ]:
solution.get('Ft')

## Testing Colony

In [ ]:
import pandas as pd
import random


# df = pd.DataFrame([(
#     ant,
#     *[random.randint(1,j) for i,j in PSystem.opzs.itertuples()]
#     ) for ant in range(1, Colony.n_ants + 1)
# ]).rename(columns = {0: 'ant'}, inplace = False).set_index('ant')